In [ ]:
import regex as re
import pandas as pd
from tqdm import tqdm

In [ ]:
f=open('/home/dell/sid/sys_data/spark/spark_logpai.log')
lines=f.readlines()

In [ ]:
split_pt=re.compile(r'([,\s$@<>\[\]\'"=]|\((?!\))|(?<!\()\)|[:\.](?=\s))')
hm_pt=re.compile(r'(^(?:\S+\s){4})(.*)')

In [ ]:
header=[]
msg=[]
for line in lines:
    if line[:3] not in set(['15/','16/','17/']):
        continue
    h,m=hm_pt.match(line).groups()
    
    header.append(h)
    msg.append(m)

In [ ]:
print(len(header))
print(header[0])
print(msg[0])

In [ ]:
header_tokens=[]
for h in header:
    tokens=[token for token in re.split(r'([\s$@<>\[\]\'"=]|\((?!\))|(?<!\()\)|[:\.,](?=\s)|(?<=\b[a-zA-Z]+):|:(?=[a-zA-Z]+\b))',h) if token!='' and token!=None]
    header_tokens.append(tokens)

In [ ]:
msg_tokens=[]
for m in msg:
    tokens=[token for token in split_pt.split(m) if token!='' and token!=None]
    msg_tokens.append(tokens)

In [ ]:
pd.DataFrame({'header_tokens':header_tokens,'msg_tokens':msg_tokens}).to_csv('tokens.csv')

In [ ]:
header_anno=[]
for tokens in header_tokens:
    anno=['O']*len(tokens)
    anno[0],anno[2],anno[4],anno[6]='B-DATE','B-TIME','B-LVL','B-CLS'

    header_anno.append(anno)

In [ ]:
list(zip(header_tokens[0],header_anno[0]))

In [ ]:
msg_anno=[]
for tokens in msg_tokens:
    anno=['O']*len(tokens)
    msg_anno.append(anno)

In [ ]:
re_date=r'\d{2}\/\d{2}\/\d{4}'
re_time=r'^\d{2}:\d{2}$'

re_obj=r'[a-zA-Z]+-\d{1,3}$'
re_cls=r'(\.([A-Z]+[a-z]*)+)|([A-Z]+[a-z]+){2,}$'
re_func=r'\S*\(\)$'
re_path=r'^\'?\/[a-zA-Z]'
re_ip=r'(?:[0-9]{1,3}\.){3}[0-9]{1,3}'

re_host=r':\d{3,5}$'
re_url=r'[a-z]+:\/\/\S*'


In [ ]:
def sub_tag(ntokens,ntags, patt_tag_list):

    def _sub_tag(ntokens,ntags, patt, target_tag):
        for tokens,tags in zip(ntokens,ntags):
            for i in range(len(tokens)):
                if re.search(patt,tokens[i]):
                    tags[i]=target_tag

    for patt,tag in tqdm(patt_tag_list):
        _sub_tag(ntokens,ntags,patt,tag)

In [ ]:
sub_tag(msg_tokens,msg_anno,
    [(re_date,'B-DATE'),
    (re_time,'B-TIME'),
    (re_cls,'B-CLS'),
    (re_func,'B-FUNC'),
    (re_path,'B-PATH'),
    (re_ip,'B-IP'),
    (re_host,'B-HOST'),
    (re_url,'B-URL')])

In [ ]:
list(zip(msg_tokens[0],msg_anno[0]))

In [ ]:
with open('./header_check.txt','w') as f:
    for tokens,tags in zip(header_tokens,header_anno):
        for token,tag in zip(tokens,tags):
            f.write(token+' '+tag+'\n')
        f.write('\n')

In [ ]:
with open('./msg_check.txt','w') as f:
    for tokens,tags in zip(msg_tokens,msg_anno):
        for token,tag in zip(tokens,tags):
            f.write(token+' '+tag+'\n')
        f.write('\n')

In [ ]:
df=pd.DataFrame({
    'tokens':[h+m for h,m in zip(header_tokens,msg_tokens)],
    'tags':[h+m for h,m in zip(header_anno,msg_anno)],
    'raw_log':[h+m for h,m in zip(header,msg)]
    })
df.to_csv('./spark_ds.csv')

In [1]:
import regex as re

In [2]:
with open('spark_ds.csv','r') as f:
    lines=f.readlines()

In [4]:
lines=[re.sub('\'<O>\'','\'O\'',line) for line in lines]
with open('spark_ds_new.csv','w') as f:
    f.writelines(lines)